## Import Python Libraries

In [3]:
# for generating random variables
import numpy as np

# for plotting
import matplotlib.pyplot as plt 

# for managing data
import pandas as pd

# for plotting
import seaborn as sns 

# for t-tests and ANOVA
import scipy.stats as stats

# for learning algorithms
import sklearn

# for encoding categorical variables
import sklearn.preprocessing as pre

# for splitting into training and test sets
import sklearn.model_selection as mod

%matplotlib inline

## Load the Dataset into Pandas

In [4]:
# read in opel_corsa_01 csv file
df = pd.read_csv("opel_corsa_01.csv", sep=";")
# display the first 5 rows of the dataframe
df.head()

,Unnamed: 0,AltitudeVariation,VehicleSpeedInstantaneous,VehicleSpeedAverage,VehicleSpeedVariance,VehicleSpeedVariation,LongitudinalAcceleration,EngineLoad,EngineCoolantTemperature,ManifoldAbsolutePressure,EngineRPM,MassAirFlow,IntakeAirTemperature,VerticalAcceleration,FuelConsumptionAverage,roadSurface,traffic,drivingStyle
0,59,-2.299988,25.670519,13.223501,121.592690,-2.476980,0.3555,4.705883,68,106,1796,15.81,24,-0.1133,19.497335,SmoothCondition,LowCongestionCondition,EvenPaceStyle
1,60,-2.099976,24.094259,13.638919,120.422571,-1.576260,0.4492,10.588236,68,103,1689,14.65,22,-0.1289,19.515722,SmoothCondition,LowCongestionCondition,EvenPaceStyle
2,61,-1.500000,22.743179,14.031043,118.456769,-1.351080,0.4258,27.450981,68,103,1599,11.85,21,-0.1328,19.441765,SmoothCondition,LowCongestionCondition,EvenPaceStyle
3,62,0.100037,22.292820,14.171073,117.571308,-0.450359,0.4140,24.313726,69,104,1620,12.21,20,-0.0859,19.388769,SmoothCondition,LowCongestionCondition,EvenPaceStyle
4,63,0.099976,23.643900,14.328954,117.074149,1.351080,0.3945,20.000000,69,104,1708,11.91,21,-0.0664,19.301638,SmoothCondition,LowCongestionCondition,EvenPaceStyle


In [5]:
# check for any missing values in the dataset - a clean dataset is required for analysis
df.isnull().sum()

Unnamed: 0                   0
AltitudeVariation            0
VehicleSpeedInstantaneous    0
VehicleSpeedAverage          0
VehicleSpeedVariance         0
VehicleSpeedVariation        0
LongitudinalAcceleration     0
EngineLoad                   0
EngineCoolantTemperature     0
ManifoldAbsolutePressure     0
EngineRPM                    0
MassAirFlow                  0
IntakeAirTemperature         0
VerticalAcceleration         0
FuelConsumptionAverage       0
roadSurface                  0
traffic                      0
drivingStyle                 0
dtype: int64

There are no missing values

In [6]:
# check the data type of all the variables
df.dtypes

Unnamed: 0                     int64
AltitudeVariation            float64
VehicleSpeedInstantaneous    float64
VehicleSpeedAverage          float64
VehicleSpeedVariance         float64
VehicleSpeedVariation        float64
LongitudinalAcceleration     float64
EngineLoad                   float64
EngineCoolantTemperature       int64
ManifoldAbsolutePressure       int64
EngineRPM                      int64
MassAirFlow                  float64
IntakeAirTemperature           int64
VerticalAcceleration         float64
FuelConsumptionAverage       float64
roadSurface                   object
traffic                       object
drivingStyle                  object
dtype: object

A mixture of datatypes - integers, floats and objects.

In [7]:
# drop the first column, roadSurface output column and traffic output column from the dataframe
corsa1 = df.drop(columns=['Unnamed: 0', 'drivingStyle', 'traffic'], axis=1)
# display the first 5 rows of corsa1
corsa1.head()

,AltitudeVariation,VehicleSpeedInstantaneous,VehicleSpeedAverage,VehicleSpeedVariance,VehicleSpeedVariation,LongitudinalAcceleration,EngineLoad,EngineCoolantTemperature,ManifoldAbsolutePressure,EngineRPM,MassAirFlow,IntakeAirTemperature,VerticalAcceleration,FuelConsumptionAverage,roadSurface
0,-2.299988,25.670519,13.223501,121.592690,-2.476980,0.3555,4.705883,68,106,1796,15.81,24,-0.1133,19.497335,SmoothCondition
1,-2.099976,24.094259,13.638919,120.422571,-1.576260,0.4492,10.588236,68,103,1689,14.65,22,-0.1289,19.515722,SmoothCondition
2,-1.500000,22.743179,14.031043,118.456769,-1.351080,0.4258,27.450981,68,103,1599,11.85,21,-0.1328,19.441765,SmoothCondition
3,0.100037,22.292820,14.171073,117.571308,-0.450359,0.4140,24.313726,69,104,1620,12.21,20,-0.0859,19.388769,SmoothCondition
4,0.099976,23.643900,14.328954,117.074149,1.351080,0.3945,20.000000,69,104,1708,11.91,21,-0.0664,19.301638,SmoothCondition


In [8]:
# count the number of categories in drivingStyle variable
pd.value_counts(corsa1.roadSurface)

SmoothCondition    6873
UnevenCondition     165
Name: roadSurface, dtype: int64

The roadSurface variable has two categories.

In [10]:
# one hot encode the roadSurface column and add two new columns to corsa1 dataframe
#pd.get_dummies(corsa1['roadSurface'], prefix='roadSurface')

pd.get_dummies(corsa1, columns=["roadSurface"]).head()

,AltitudeVariation,VehicleSpeedInstantaneous,VehicleSpeedAverage,VehicleSpeedVariance,VehicleSpeedVariation,LongitudinalAcceleration,EngineLoad,EngineCoolantTemperature,ManifoldAbsolutePressure,EngineRPM,MassAirFlow,IntakeAirTemperature,VerticalAcceleration,FuelConsumptionAverage,roadSurface_SmoothCondition,roadSurface_UnevenCondition
0,-2.299988,25.670519,13.223501,121.592690,-2.476980,0.3555,4.705883,68,106,1796,15.81,24,-0.1133,19.497335,1,0
1,-2.099976,24.094259,13.638919,120.422571,-1.576260,0.4492,10.588236,68,103,1689,14.65,22,-0.1289,19.515722,1,0
2,-1.500000,22.743179,14.031043,118.456769,-1.351080,0.4258,27.450981,68,103,1599,11.85,21,-0.1328,19.441765,1,0
3,0.100037,22.292820,14.171073,117.571308,-0.450359,0.4140,24.313726,69,104,1620,12.21,20,-0.0859,19.388769,1,0
4,0.099976,23.643900,14.328954,117.074149,1.351080,0.3945,20.000000,69,104,1708,11.91,21,-0.0664,19.301638,1,0


In [12]:
# check the data type of all the variables
corsa1.dtypes

AltitudeVariation            float64
VehicleSpeedInstantaneous    float64
VehicleSpeedAverage          float64
VehicleSpeedVariance         float64
VehicleSpeedVariation        float64
LongitudinalAcceleration     float64
EngineLoad                   float64
EngineCoolantTemperature       int64
ManifoldAbsolutePressure       int64
EngineRPM                      int64
MassAirFlow                  float64
IntakeAirTemperature           int64
VerticalAcceleration         float64
FuelConsumptionAverage       float64
roadSurface                   object
dtype: object

### Descriptive Statistics
Now that the dataset has been loaded into a Panda's dataframe, I will further analyse the dataset to learn more about it. The Panda's describe function produces the descriptive statistics on the dataset. The mean, standard deviation, maximum number, minimum number, 25% quartile range (median), 50% quartile range and 75% quartile range are produced for each of the 14 numeric variables.

In [11]:
# descriptive statistics are obtained using the describe function. Adding the .T swaps the rows and columns.
corsa1.describe().T

,count,mean,std,min,25%,50%,75%,max
AltitudeVariation,7038.0,-0.675845,1.691601,-9.200012,-1.500000,-0.399963,0.100037,5.200012
VehicleSpeedInstantaneous,7038.0,36.428319,32.901312,0.000000,8.782019,29.273399,54.043198,124.749725
VehicleSpeedAverage,7038.0,36.723932,29.366391,0.000000,16.698035,28.312631,47.595544,121.330733
VehicleSpeedVariance,7038.0,213.004353,205.717663,0.000000,54.333652,144.864363,299.377339,1051.789888
VehicleSpeedVariation,7038.0,-0.029563,2.390997,-17.789218,-0.900722,0.000000,0.900721,12.384899
LongitudinalAcceleration,7038.0,0.143530,0.744697,-2.380000,-0.339800,0.140800,0.683600,2.360000
EngineLoad,7038.0,26.487416,19.462750,0.000000,13.725491,25.490196,34.901962,100.000000
EngineCoolantTemperature,7038.0,77.924979,7.076616,40.000000,79.000000,80.000000,81.000000,85.000000
ManifoldAbsolutePressure,7038.0,116.234157,20.660674,98.000000,102.000000,109.000000,122.000000,252.000000
EngineRPM,7038.0,1569.145354,551.406613,752.000000,936.000000,1659.500000,2033.000000,3104.000000


It is difficult to obtain a clear picture of the dataset using the basic raw descriptive statistics - it is not very informative in this state. Visualisation tools provide a much better way to analyse the dataset and will give a better understanding of the data.

### Visualisations

Visual tools such as histograms, boxplots and scatterplots make it easier to visualise the distribution of variables.



In [ ]:
list(corsa1.columns)

In [ ]:
# run the Panda's corr function on the corsa1 dataframe - method Pearson
corsa1.corr(method='spearman')